In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import sys
sys.path.append('utils/')

import keras
from keras import optimizers

# use non standard flow_from_directory
from image_preprocessing_ver2 import ImageDataGenerator
# it outputs y_batch that contains onehot targets and logits
# logits came from resnet

from keras.applications.mobilenet import preprocess_input
from keras.models import Model
from keras.layers import Lambda, concatenate, Activation
from keras.losses import categorical_crossentropy as logloss
from keras.metrics import categorical_accuracy, top_k_categorical_accuracy
from keras import backend as K

from mobilenet import get_mobilenet

Using TensorFlow backend.


In [3]:
data_dir = '/home/ubuntu/data/'

In [4]:
train_logits = np.load(data_dir + 'train_logits.npy')[()]
val_logits = np.load(data_dir + 'val_logits.npy')[()]

In [5]:
data_generator = ImageDataGenerator(
    data_format='channels_last',
    preprocessing_function=preprocess_input
)

train_generator = data_generator.flow_from_directory(
    data_dir + 'train', train_logits,
    target_size=(224, 224),
    batch_size=64
)

val_generator = data_generator.flow_from_directory(
    data_dir + 'val', val_logits,
    target_size=(224, 224),
    batch_size=64
)

Found 25600 images belonging to 256 classes.
Found 5120 images belonging to 256 classes.


In [17]:
temperature = 15
lambda_const = 0.01

In [18]:
model = get_mobilenet()
model.layers.pop()

for layer in model.layers[:-1]:
    layer.trainable = False
    
model.layers[-1].kernel_regularizer = keras.regularizers.l2(1e-4)

logits = model.layers[-1].output
probabilities = Activation('softmax')(logits)

logits_T = Lambda(lambda x: x/temperature)(logits)
probabilities_T = Activation('softmax')(logits_T)

output = concatenate([probabilities, probabilities_T])
model = Model(model.input, output)

In [19]:
def knowledge_distillation_loss(y_true, y_pred):    
    y_true, y_soft = y_true[:, :256], y_true[:, 256:]
    y_soft = K.softmax(y_soft/temperature)
    y_pred, y_pred_soft = y_pred[:, :256], y_pred[:, 256:]    
    return lambda_const*logloss(y_true, y_pred) + logloss(y_soft, y_pred_soft)

In [20]:
def accuracy(y_true, y_pred):
    y_true = y_true[:, :256]
    y_pred = y_pred[:, :256]
    return categorical_accuracy(y_true, y_pred)

In [21]:
def top_k_accuracy(y_true, y_pred):
    y_true = y_true[:, :256]
    y_pred = y_pred[:, :256]
    return top_k_categorical_accuracy(y_true, y_pred)

In [22]:
model.compile(
    optimizer=optimizers.RMSprop(lr=1e-3), 
    loss=knowledge_distillation_loss, metrics=[accuracy, top_k_accuracy]
)

In [23]:
model.fit_generator(
    train_generator, 
    steps_per_epoch=150, epochs=5, verbose=1,
    validation_data=val_generator, validation_steps=24, 
    max_queue_size=10, workers=1, use_multiprocessing=False
)

Epoch 1/5
150/150 [==============================] - 47s - loss: 5.5728 - accuracy: 0.4082 - top_k_accuracy: 0.6171 - val_loss: 5.5582 - val_accuracy: 0.6152 - val_top_k_accuracy: 0.8509
Epoch 2/5
150/150 [==============================] - 47s - loss: 5.5529 - accuracy: 0.7343 - top_k_accuracy: 0.9119 - val_loss: 5.5529 - val_accuracy: 0.7214 - val_top_k_accuracy: 0.9043
Epoch 3/5
150/150 [==============================] - 47s - loss: 5.5491 - accuracy: 0.8141 - top_k_accuracy: 0.9421 - val_loss: 5.5508 - val_accuracy: 0.7500 - val_top_k_accuracy: 0.9173
Epoch 4/5
150/150 [==============================] - 47s - loss: 5.5460 - accuracy: 0.8860 - top_k_accuracy: 0.9702 - val_loss: 5.5504 - val_accuracy: 0.7663 - val_top_k_accuracy: 0.9238
Epoch 5/5
150/150 [==============================] - 47s - loss: 5.5454 - accuracy: 0.8948 - top_k_accuracy: 0.9742 - val_loss: 5.5504 - val_accuracy: 0.7604 - val_top_k_accuracy: 0.9180


In [24]:
val_generator_no_shuffle = data_generator.flow_from_directory(
    data_dir + 'val', val_logits,
    target_size=(224, 224),
    batch_size=64, shuffle=False
)

Found 5120 images belonging to 256 classes.


In [25]:
model.evaluate_generator(val_generator_no_shuffle, 80)

[5.5500422775745388, 0.76796874999999998, 0.9228515625]